# Analyze Received Long Multi Tone Calibration WAV File
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y %Z", localtime (time())))
%plot --format svg
format free
pkg load signal

Last updated: Monday 13 January 2020 UTC

In [2]:
inputFilename = "/mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav"

inputFilename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav


In [3]:
audioinfo(inputFilename)

ans =

  scalar structure containing the fields:

    Filename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav
    CompressionMethod = 
    NumChannels = 2
    SampleRate = 62500
    TotalSamples = 1.40625e+07
    Duration = 225
    BitsPerSample = -1
    BitRate = -1
    Title = 
    Artist = 
    Comment = 



In [4]:
[SamplesIQRV, sampleRate] = audioread(inputFilename);
size(SamplesIQRV)
sampleRate

ans =

 1.40625e+07 2


sampleRate = 62500


In [5]:
SamplesRV=SamplesIQRV(:,1)+i*SamplesIQRV(:,2);
size(SamplesRV)

ans =

 1.40625e+07 1




In [6]:
# plot(abs(SamplesRV));

In [7]:
numPeriods = rows(SamplesRV)/sampleRate;
period = numPeriods;
TshortRV = [0:1/sampleRate:1](:);
TfullRV = [0:1/sampleRate:period](:);

In [8]:
RaisedCosineRV = (1 - cos(2*pi*TshortRV))/2;
size(RaisedCosineRV)

ans =

 62501 1




In [9]:
SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
size(SigSec0RV)

ans =

 62501 1




In [10]:
# plot(abs(SigSec0RV));

In [11]:
FftSec0RV = abs(fftshift(fft(SigSec0RV, sampleRate*10 + 1)));
FreqSec0RV = [linspace(-sampleRate/2, sampleRate/2, rows(FftSec0RV))](:);

In [12]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([100 800], "tic")

In [13]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% idx = find(FftSec0RV(:) == max(FftSec0RV))
% FftSec0RV(idx)
% FreqSec0RV(idx)
% axis([ans-4 ans+4], "tic")

In [14]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([235-4 235+4], "tic")

In [15]:
find(FreqSec0RV(:) == 230)

ans = 314801


In [16]:
find(FreqSec0RV(:) == 245)

ans = 314951


In [17]:
idx = find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))
FftSec0RV(idx)
FreqSec0RV(idx)

idx = 314855
ans = 141.949
ans = 235.4


In [18]:
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))) - ...
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV)))

ans = -521


In [19]:
721 - 521

ans = 200


In [28]:
fracSeconds = 1;
indexIncr = (rows(TshortRV)-1)/fracSeconds;
pilotFreq = 721;
Response = [];
# Use a most positive initial reference freq because probe freq starts low and increases
refFreq = sampleRate * 2;
for startIndex = 1:indexIncr:rows(SamplesRV)-rows(RaisedCosineRV)-1
  time = TfullRV(startIndex);
  # SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
  SigLoopRV = SamplesRV(startIndex:startIndex+rows(RaisedCosineRV)-1) .* RaisedCosineRV;
  FftLoopRV = fftshift(fft(SigLoopRV, sampleRate*10 + 1));
  FftLoopAbsRV = abs(FftLoopRV);
  # max(FftLoopAbsRV)
  # FftLoopPolCV = cart2pol(real(FftLoopRV), imag(FftLoopRV)); # rows of [theta, r]
  # FftLoopPolCV(1,:)
  FreqLoopRV = linspace(-sampleRate/2, sampleRate/2, rows(FftLoopRV));
  freqMaxPeak = FreqLoopRV(find(FftLoopAbsRV(:) == max(FftLoopAbsRV)));
  [PeaksCV idxPeaksCV] = findpeaks(FftLoopAbsRV, "MinPeakHeight", 50);
  if (rows(idxPeaksCV) == 1)
    idxPeaksCV = [idxPeaksCV; idxPeaksCV];
  endif
  FreqPeaksLoopRV = FreqLoopRV(idxPeaksCV);
  # Find the reference frequency based on minimum difference from the last iteration
  FreqDiff = abs(FreqPeaksLoopRV .- refFreq);
  minFreqDiff = min(FreqDiff);
  refFreqIdx = find(FreqDiff(:) == minFreqDiff, 1);
  probeFreqIdx = rows(idxPeaksCV) + 1 - refFreqIdx;
  # Determine corrected (expected) frequencies
  corrFreqLoopRV = FreqPeaksLoopRV .- FreqPeaksLoopRV(refFreqIdx) .+ pilotFreq;
  # Get ratio of response
  FftLoopRV(idxPeaksCV(probeFreqIdx))/FftLoopRV(idxPeaksCV(refFreqIdx));
  [phase, mag] = cart2pol(real(ans), imag(ans)); # rows of [theta, r]
  Response = [Response; time, rows(idxPeaksCV), corrFreqLoopRV(probeFreqIdx), phase*180/pi, 20*log10(mag)];
  
  # Compute the new reference frequency
  refFreq = FreqPeaksLoopRV(refFreqIdx);
endfor